# How to Rug a Bitcoin Stamp

## aka how to sweep Counterparty bare multisig outputs and reclaim your sats

In [ ]:
!pip install pybtc

In [ ]:
import requests
from pybtc import *

In [ ]:
INPUT_TRANSACTION_ID = 'txid that created the bare multisig outputs'
PRIVATE_KEY = 'Private key can be obtained via freewallet by going to Address -> View Private Key'
ADDRESS_TO_SEND_TO = 'address you want your reclaimed sats sent to'
FEE_SATS = 50000  # Adjust me to set your mining fee

In [ ]:
# Retreive the bare multisig input transaction via mempool.space API
input_tx = requests.get(f"https://mempool.space/api/tx/{INPUT_TRANSACTION_ID}").json()

In [ ]:
# Construct a new transaction using the bare multisig outputs as inputs
input_sats = 0
tx = Transaction()
#tx.rbf = True
for n, prevout in enumerate(input_tx['vout']):
    if prevout['scriptpubkey_type'] == 'multisig':
        tx.add_input(input_txid, n)
        input_sats += prevout['value']

In [ ]:
# Calculate the output sats with the remainder going to mining fee
output_sats = input_sats - FEE_SATS
# Add an output to send the sats to our address
tx.add_output(output_sats, address=ADDRESS_TO_SEND_TO)

In [ ]:
# Sign each of the inputs using the private key from the counterparty wallet which created
# the bare multisig outputs. By default counterparty locks the inputs using the pubkey from the
# first input as the 1 valid key in it's 1 of 3 bare multisig outputs
# https://docs.counterparty.io/docs/develop/api/#advanced-create_-parameters
for n, vin in tx["vIn"].items():
    tx.sign_input(n, private_key=[PRIVATE_KEY], script_pub_key=input_tx['vout'][n]['scriptpubkey'])

In [ ]:
# Serialize the transaction
# This transcation can now be copy and pasted into Sparrow Wallet for inspection and broadcast using the
# File -> Open Transaction -> From Text feature
tx.serialize()